# Text-to-SQL Chain Evaluations 2

In [1]:
import getpass
import os
import pandas as pd
import json

import openai
import anthropic

from langchain.globals import set_debug
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import tool
from langchain.agents import AgentExecutor
from langchain_community.agent_toolkits import SQLDatabaseToolkit

from sqlalchemy import create_engine
from urllib.parse import quote_plus

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)

#set environmental variable for Anthropic API key
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Calude key here: ")

#set environmental variable for OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("GPT key here: ")

Calude key here:  ········
GPT key here:  ········


In [2]:
test_set = pd.read_csv('src/test_set.csv')
questions = test_set['question']
ground_truths = test_set['answer']

def run_test(model, model_type):
    answers = []

    for query in questions:
        response = model.invoke(query)
        if model_type == "text2sql":
            answers.append(response)
        else:
            answers.append(response.content)
        
    # To dict
    data = {
        "question": questions,
        "answer": answers,
        "ground_truths": ground_truths
    }
    
    dataset = pd.DataFrame.from_dict(data)
    return dataset

In [8]:
#create baseline model chain for evaluation
class baseline_chain:
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = self.create_baseline_model()
        
    def create_baseline_model(self):
        if "claude" in self.model_name:
            baseline_llm = ChatAnthropic(model = self.model_name, temperature = 0)
        else:
            baseline_llm = ChatOpenAI(model_name = self.model_name, temperature = 0)
    
        baseline_prompt = ChatPromptTemplate.from_messages([
            ("system", "Act as a WNBA expert and answer this user question"),
            ("user", "{input}"),
        ])
        
        baseline_chain = baseline_prompt | baseline_llm
    
        return baseline_chain

    def invoke(self, query):
        return self.model.invoke({"input": query})

In [22]:
#create a Pydantic Base Model
class Table(BaseModel):
    """Table in SQL database."""
    name: str = Field(description = "Name of table in SQL database.")

#create table chain
class TableChain:
    def __init__(self, model_name):
        self.model_name = model_name
        self.table_chain = self.create_table_chain()

    def get_table_details(self):
        file_paths = [
            "src/wnba_nba_pbp_data_dict.json",
            "src/wnba_player_box.json",
            "src/wnba_player_info.json",
            "src/wnba_schedule.json",
            "src/wnba_teambox.json"
        ]
        
        table_details = ""
    
        for file_path in file_paths:
            
            #load table names and descriptions from json
            f = open(file_path)
            table_dict = json.load(f)
            
            #retrieve table names and descriptions and compile into string
            table_details = table_details + "Table Name:" + table_dict['table_name'] + "\n" \
            + "Table Description:" + table_dict['table_description']
    
            for col in table_dict['values']:
                table_details = table_details + "\n" + "Column Name:" + col['column_name'] + "\n" \
                + "Column Description:" + col['column_description'] + "\n" \
                + "COlumn Type:" + col['column_type']
            
            table_details = table_details + "\n\n"
        
        return table_details

    def create_table_chain(self):
        
        table_details = self.get_table_details()
        
        #create table chain prompt
        table_prompt_system = f"""Refer the Above Context and Return the names of SQL Tables that MIGHT be relevant to the above context\n\n 
        The tables are:
        
        {table_details}
         """
        
        table_details_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", table_prompt_system),
                ("human", "{input}"),
            ]
        )
        
        #set up LLM
        if "claude" in self.model_name:
            table_chain_llm = ChatAnthropic(model = self.model_name, temperature = 0)
        else:
            table_chain_llm = ChatOpenAI(model_name = self.model_name, temperature = 0)
        table_chain_llm_wtools = table_chain_llm.bind_tools([Table])
        output_parser = PydanticToolsParser(tools=[Table])

        #create table chain
        table_chain = table_details_prompt | table_chain_llm_wtools | output_parser

        return table_chain

    def invoke(self, query):
        return self.table_chain.invoke(query)

#create text2sql chain
class QAEngine:
    def __init__(self, model_name):
        self.model_name = model_name
        self.db = self.load_db()
        self.query_examples = self.load_query_examples()
        self.table_chain = TableChain(model_name)
        self.text_sql_agent, self.sql_tools = self.create_text_sql_agent()

    def load_db(self):
        #database info
        AWS_REGION = "us-east-1"
        SCHEMA_NAME = "wnba_db"
        S3_STAGING_DIR = "s3://wnbadata/"
        
        connect_str = "awsathena+rest://athena.{region_name}.amazonaws.com:443/{schema_name}?s3_staging_dir={s3_staging_dir}"
        
        #connect to AWS athena
        engine = create_engine(connect_str.format(
                region_name = AWS_REGION,
                schema_name = SCHEMA_NAME,
                s3_staging_dir = quote_plus(S3_STAGING_DIR)
        ))
        
        #create SQL db object
        return SQLDatabase(engine)

    def load_query_examples(self):
        #load query examples from json
        f = open("src/query_example.json")
        query_examples_json = json.load(f)
        
        return query_examples_json["query_examples"]

    def create_text_sql_agent(self):
        #set up LLM
        if "claude" in self.model_name:
            llm = ChatAnthropic(model = self.model_name, temperature = 0)
        else:
            llm = ChatOpenAI(model_name = self.model_name, temperature= 0)
        #utilize langchain agent toolkit: SQL toolkit
        toolkit = SQLDatabaseToolkit(db = self.db, llm = llm)
        tools = toolkit.get_tools()
        llm_with_tools = llm.bind_tools(tools)
        
        #create few shot prompt with query examples
        example_prompt = ChatPromptTemplate.from_messages(["User input: {input}\nSQL query: {query}"])
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            examples = self.query_examples,
            example_prompt = example_prompt,
        )
        
        #create prompt for agent
        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """You are an agent designed to interact with a SQL database and a WNBA expert to answer questions about the WNBA.
                Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
                If applicable, return the statistical numbers as well.
                You should also leverage your pre-existing knowledge of WNBA rules, statistics, teams, players, and history to understand and interpret user questions and your answer accurately.
                Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
                You can order the results by a relevant column to return the most interesting examples in the database.
                Never query for all the columns from a specific table, only ask for the relevant columns given the question.
                You have access to tools for interacting with the database.
                Base your final answer on the information returned by these tools, combined with your existing knowledge of the WNBA.
                You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
        
                When referring to a specific game, DO NOT just provide the game ID. You MUST also provide the two teams involved in the game.
                
                DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
        
                DO NOT give me the SQL query.
                
                You MUST always execute the query and use the result to contruct the final answer.
                
                To start you should ALWAYS look at the tables in the database to see what you can query.
                Do NOT skip this step.
                Then you should query the schema of the most relevant tables.
                Here is the relevant table info, but feel free to use other tables you deem necessary: {table_names_to_use}.""",
            ),
            few_shot_prompt,
            ("user", "{input}"),
            #message placeholder for storing intermediate steps
            MessagesPlaceholder(variable_name = "agent_scratchpad"),
        ])

        #create agent
        agent = (
            {
                "input": lambda x: x["input"],
                "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                    x["intermediate_steps"]
                ),
                "table_names_to_use": lambda x: x["table_names_to_use"],
            }
            | prompt
            | llm_with_tools
            | OpenAIToolsAgentOutputParser()
        )
        
        return agent, tools

    def invoke(self, query):
        response = None
        max_iteration = 10
        error_count = 0
        count = 0
        agent_executor = AgentExecutor(agent = self.text_sql_agent, tools = self.sql_tools, verbose = False, return_intermediate_steps = True, max_iterations = max_iteration)
        while response is None and count <= 5:
            try:
                response = agent_executor.invoke({
                "input": query,
                #passing result of table chain to text2sql agent
                "table_names_to_use": self.table_chain.invoke(query)
                })
            except (openai.BadRequestError, openai.APIError, anthropic.APIStatusError) as e:
                if error_count == 0:
                    max_iteration = 3
                elif error_count == 1:
                    max_iteration = 1
                agent_executor = AgentExecutor(agent = self.text_sql_agent, tools = self.sql_tools, verbose = False, return_intermediate_steps = True, max_iterations = max_iteration)
                error_count += 1
                count += 1                
            else:
                if 'SQL query' in response['output'] or 'Agent stopped' in response['output']:
                    response = None
                    count += 1
                    
        if response is not None:
            return response['output']
        else:
            return "no response"

### Baseline Model (GPT3.5 Turbo)

In [9]:
GPT35T_baseline = baseline_chain("gpt-3.5-turbo-0125")
GPT35T_baseline_results = run_test(GPT35T_baseline, "baseline")
GPT35T_baseline_results

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"As of now, the player who scored the most points in a single game between 2018 and 2024 is Liz Cambage. She set a WNBA record by scoring 53 points in a game for the Dallas Wings against the New York Liberty on July 17, 2018. This performance by Cambage surpassed the previous record of 51 points set by Riquna Williams in 2013.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"The Seattle Storm had the highest average points per game in the 2021 WNBA season, averaging 87.4 points per game. Led by superstars like Breanna Stewart and Jewell Loyd, the Storm had a high-powered offense that consistently put up big numbers throughout the season.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"During the 2023 WNBA season, there were a total of 12 games that ended in overtime. Overtime games are always exciting to watch as they add an extra level of intensity and drama to the competition.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"In the 2020 WNBA season, the player with the highest free throw percentage (minimum 25 free throws attempted) was Allie Quigley of the Chicago Sky. Quigley shot an impressive 95.7% from the free throw line, making 45 out of 47 attempts. She was known for her consistency and accuracy from the charity stripe throughout the season.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"As of the end of the 2022 WNBA regular season, the Seattle Storm had a record of 24 wins and 10 losses, finishing in second place in the Western Conference standings. The Storm had a successful season and qualified for the playoffs.","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"The Washington Mystics had the longest winning streak in the 2019 WNBA season, with a 12-game winning streak that spanned from July 30 to September 6. This streak helped propel them to the best record in the league and ultimately win the WNBA Championship that year.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","Between 2018 and 2024, there were a total of 14 triple-doubles recorded in the WNBA. The player with the most triple-doubles during this time period was Sabrina Ionescu of the New York Liberty, who recorded 6 triple-doubles. Other players who recorded triple-doubles during this time frame include Courtney Vandersloot, Liz Cambage, and Breanna Stewart. Triple-doubles are a rare and impressive feat in the WNBA, so achieving one is a significant accomplishment.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"In the 2024 WNBA season, the Seattle Storm had the best defense in terms of opponent points per game. They allowed an average of 74.2 points per game, which was the lowest in the league. Led by defensive stalwarts like Breanna Stewart and Jewell Loyd, the Storm's defensive prowess played a key role in their success that season.",Connecticut Sun (opponents held to 71.08 PPG)
8,Identify the game with the highest combined score (both teams) from 2018 to 2024.,"The game with the highest combined score in WNBA history occurred on July 8, 2018, between the Dallas Wings and the Washington Mystics. The final score was 108-105 in favor of the Mystics, resulting in a combined score of 213 points. This game set a new record for the highest combined score in WNBA history.","07/12/2022, Phoenix Mercury @ Minnesota Lynx (225 points combined)"
9,Which player played the most minutes in the 2021 season?,"In the 2021 WNBA season, the player who played the most minutes was 

### Baseline Model (GPT4 Turbo)

In [10]:
GPT4T_baseline = baseline_chain("gpt-4-turbo")
GPT4T_baseline_results = run_test(GPT4T_baseline, "baseline")
GPT4T_baseline_results

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"As of my last update in 2023, the record for the most points scored in a single WNBA game between 2018 and 2023 is held by Liz Cambage. She scored 53 points for the Dallas Wings in a game against the New York Liberty on July 17, 2018. This performance set the record for the most points in a single WNBA game. Please check the latest statistics or sources for any updates beyond 2023.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"In the 2021 WNBA season, the Connecticut Sun had the highest average points per game. They led the league offensively, showcasing a strong and consistent performance throughout the season.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"As of my last update, I don't have the specific data for the 2023 WNBA season, including the number of games that ended in overtime. For the most accurate and current information, I recommend checking the WNBA's official website or reliable sports news sources that track and update WNBA season statistics in real time.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"In the 2020 WNBA season, the player with the highest free throw percentage (with a minimum of 25 free throws attempted) was Brionna Jones of the Connecticut Sun. She achieved a free throw percentage of 90.9%, making 30 out of 33 attempts.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"In the 2022 WNBA season, the Seattle Storm finished with a win-loss record of 22-14.","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"In the 2019 WNBA season, the longest winning streak, including the postseason, was held by the Washington Mystics. The Mystics had a 10-game winning streak from August 25 to September 19, 2019. This streak was crucial in propelling them towards their first WNBA Championship. They finished the regular season with a 26-8 record and continued their strong performance through the playoffs.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","As of my last update in 2023, I don't have the complete data for 2024. However, up to 2023, triple-doubles in the WNBA were still a rare feat. Notably, Sabrina Ionescu and Courtney Vandersloot were among the players who had recorded multiple triple-doubles in their careers.\n\nSabrina Ionescu, who was drafted by the New York Liberty in 2020, quickly made a name for herself by achieving several triple-doubles, showcasing her versatility and skill in scoring, assists, and rebounds. Courtney Vandersloot, playing for the Chicago Sky, also had multiple triple-doubles, emphasizing her exceptional playmaking abilities.\n\nFor the exact number of triple-doubles recorded between 2018 and 2023 and the player with the most triple-doubles, I recommend checking the latest statistics from the WNBA's official website or other reliable sports statistics websites, as these figures are subject to change with ongoing seasons and games.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"I'm sorry for any confusion, but as of my last update in 2023, I don't have the specific statistics for the 2024 WNBA season. To find the most accurate and current information regarding which team had the best defense in the 2024 WNBA season based on opponent points, I recommend checking the latest stats on the WNBA's official website or a reliable sports statistics website like ESPN or Basketball Reference. They provide detailed and updated team stats that include defensive rankings.",Connecticut Sun (opponents h

### Baseline Model (GPT4o)

In [11]:
GPT4o_baseline = baseline_chain("gpt-4o")
GPT4o_baseline_results = run_test(GPT4o_baseline, "baseline")
GPT4o_baseline_results

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"As of my last update in October 2023, the record for the most points scored in a single WNBA game between 2018 and 2023 is held by Liz Cambage. She scored 53 points for the Dallas Wings against the New York Liberty on July 17, 2018. This performance set the record for the most points in a single game in WNBA history. If there have been any new records set between now and 2024, I would recommend checking the latest WNBA statistics or official sources for the most up-to-date information.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"In the 2021 WNBA season, the Las Vegas Aces had the highest average points per game. They led the league with an impressive offensive performance, averaging around 89.3 points per game. The Aces' high-scoring offense was a key factor in their successful season, which saw them finish with one of the best records in the league.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"In the 2023 WNBA season, there were 15 games that ended in overtime. This number reflects the competitive nature of the league, with many closely contested matchups requiring extra time to determine a winner.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"In the 2020 WNBA season, the player with the highest free throw percentage (minimum 25 free throws attempted) was Arike Ogunbowale of the Dallas Wings. She achieved an impressive free throw percentage of 89.0%. Ogunbowale's accuracy from the free-throw line was a significant aspect of her scoring prowess throughout the season.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,The Seattle Storm finished the 2022 WNBA regular season with a win-loss record of 22-14. They secured the fourth seed in the playoffs but were eventually eliminated in the semifinals by the Las Vegas Aces.,"25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"In the 2019 WNBA season, the Washington Mystics had the longest winning streak, which included both the regular season and the postseason. The Mystics, led by MVP Elena Delle Donne, were dominant throughout the season and carried their momentum into the playoffs. They ultimately won the WNBA Championship, securing their first title in franchise history. Their impressive performance included a winning streak that showcased their strength and consistency as a team.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","As of my last update in October 2023, I can provide information up to that point. Between 2018 and 2023, the WNBA saw an increase in the number of triple-doubles recorded. \n\nOne of the standout players in this regard is Sabrina Ionescu of the New York Liberty, who has been particularly notable for her ability to record triple-doubles. She set the record for the most triple-doubles in a single season in 2022.\n\nHowever, for the complete and most accurate count of triple-doubles recorded between 2018 and 2024, and to identify the player with the most triple-doubles during that period, you would need to refer to the latest WNBA statistics and records, as my data does not include the 2024 season and may not cover the entirety of 2023.\n\nFor the most up-to-date information, you can check the official WNBA website or trusted sports news sources that track WNBA statistics.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"As of my last update in October 2023, I don't have specific data on the 2024 WNBA season. However, you can typically find the most up-to-date sta

### Baseline Model (GPT4o mini)

In [13]:
GPT4omini_baseline = baseline_chain("gpt-4o-mini")
GPT4omini_baseline_results = run_test(GPT4omini_baseline, "baseline")
GPT4omini_baseline_results

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"As of October 2023, the record for the most points scored in a single WNBA game between 2018 and 2024 is held by Kelsey Plum. She scored 53 points on August 6, 2022, while playing for the Las Vegas Aces against the New York Liberty. This performance not only set a franchise record but also marked one of the highest single-game scoring outputs in WNBA history.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"In the 2021 WNBA season, the team with the highest average points per game was the Dallas Wings. They averaged 84.0 points per game throughout the season. This offensive output was a key factor in their competitive play during the season.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"During the 2023 WNBA season, there were a total of 5 games that ended in overtime. Overtime games can be quite thrilling, showcasing the competitive spirit of the league. If you have any more questions about the season or specific games, feel free to ask!",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"In the 2020 WNBA season, the player with the highest free throw percentage, with a minimum of 25 free throws attempted, was Arike Ogunbowale of the Dallas Wings. She finished the season with a free throw percentage of 92.9%. Ogunbowale's ability to convert from the charity stripe was a key aspect of her scoring and overall performance during that season.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"In the 2022 WNBA season, the Seattle Storm finished with a win-loss record of 22 wins and 14 losses. They secured the fourth seed in the playoffs but were eliminated in the first round by the Las Vegas Aces.","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"In the 2019 WNBA season, the Washington Mystics had the longest winning streak, which included both the regular season and postseason. They won 10 consecutive games, starting from the regular season and continuing through the playoffs, culminating in their victory in the WNBA Finals. This impressive run was a key factor in their success that season, leading them to their first championship in franchise history.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","As of October 2023, there have been a total of 12 triple-doubles recorded in the WNBA from 2018 to 2023. The player with the most triple-doubles during this period is Sabrina Ionescu of the New York Liberty, who has made a significant impact in the league with her all-around play. Keep in mind that the statistics for the 2024 season are not available yet, as my training only includes data up to October 2023. For the most current information, you may want to check the latest WNBA statistics or news sources.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"As of my last update in October 2023, I don't have access to data from the 2024 WNBA season. To find out which team had the best defense in terms of opponent points allowed for the 2024 season, I recommend checking the official WNBA website or sports news outlets that cover the league for the most current statistics. They typically provide detailed team stats, including defensive rankings.",Connecticut Sun (opponents held to 71.08 PPG)
8,Identify the game with the highest combined score (both teams) from 2018 to 2024.,"As of my last update in October 2023, the game with the highest combined score in WNBA history occurred on July 27, 2020, between the Atlanta Dream and the Dallas Wings. The final sco

### Baseline Model (Claude3.5 Sonnet)

In [21]:
Claude35_baseline = baseline_chain("claude-3-5-sonnet-20240620")
Claude35_baseline_results = run_test(Claude35_baseline, "baseline")
Claude35_baseline_results

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"The player who scored the most points in a single WNBA game between 2018 and 2024 is Liz Cambage, who set the WNBA single-game scoring record with 53 points on July 17, 2018, while playing for the Dallas Wings against the New York Liberty.\n\nCambage's performance was historic, as she broke the previous record of 51 points, which had been held by Riquna Williams since 2013. In addition to her 53 points, Cambage also grabbed 10 rebounds and blocked 5 shots in that game, showcasing her all-around dominance.\n\nThis record-breaking performance remains the highest single-game scoring output in WNBA history as of 2024, and it occurred within the timeframe you specified (2018-2024).",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"In the 2021 WNBA season, the Las Vegas Aces led the league in scoring, averaging 89.3 points per game. They had a high-powered offense featuring players like A'ja Wilson, Liz Cambage, and Kelsey Plum. The Aces' fast-paced style and efficient shooting helped them maintain the top spot in scoring average throughout the season.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"I apologize, but I don't have the exact number of overtime games for the 2023 WNBA season readily available. WNBA overtime statistics are not as commonly reported or easily accessible as some other league-wide stats. To get an accurate count, one would need to review the results of all games played during the 2023 season. If you need this specific information, I'd recommend checking the official WNBA website or reaching out to their statistics department directly. They would have the most accurate and up-to-date information on overtime games for any given season.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"Based on the 2020 WNBA season statistics, with a minimum of 25 free throws attempted, the player with the highest free throw percentage was Allie Quigley of the Chicago Sky.\n\nAllie Quigley shot an impressive 96.4% from the free throw line, making 53 out of 55 attempts during the 2020 season.\n\nQuigley, known for her excellent shooting, has consistently been one of the best free throw shooters in the WNBA throughout her career. Her performance in 2020 was particularly outstanding, as she nearly achieved a perfect percentage from the line.\n\nIt's worth noting that the 2020 WNBA season was shortened due to the COVID-19 pandemic, which may have affected the total number of attempts for some players. However, Quigley's efficiency remained exceptional despite the unusual circumstances of that season.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"In the 2022 WNBA season, the Seattle Storm finished with a record of 22 wins and 14 losses. This placed them fourth in the overall league standings and secured them a playoff berth. The Storm made it to the semifinals of the playoffs before being eliminated by the Las Vegas Aces, who went on to win the WNBA championship that year.","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"In the 2019 WNBA season, including both regular season and postseason games, the Washington Mystics had the longest winning streak. They won 8 consecutive games from August 8 to September 8, 2019.\n\nThis streak included:\n- The last 6 games of their regular season\n- Their first 2 playoff games (both semifinal games against the Las Vegas Aces)\n\nThe Mystics finished the regular season with a league-best 26-8 record and went on to win the WNBA Championship that year, defeating the Connecticut Sun in the Finals.\n\nIt's worth noting that while this was the longest streak including postseason games, if we consider

### Text2SQL (GPT3.5 Turbo)

In [14]:
gpt35T_text2sql = QAEngine("gpt-3.5-turbo-0125")
gpt35T_text2sql_result = run_test(gpt35T_text2sql, "text2sql")
gpt35T_text2sql_result

/var/folders/1c/ybx_8cds1ybgdg3z19ptmmk00000gn/T/ipykernel_27648/541717573.py:91: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pyathena.sqlalchemy.rest.AthenaRestDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(connect_str.format(
Failed to execute query.
Traceback (most recent call last):
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/pyathena/common.py", line 575, in _execute
    query_id = retry_api_call(
               ^^^^^^^^^^^^^^^
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/tenacity/__init__.py", line 4

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,The player who scored the most points in a single game between 2018 and 2024 was A'ja Wilson with 53 points.,A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,The team with the highest average points per game in the 2021 season was the Las Vegas Aces with an average of 89.25 points per game.,Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,There were no games that ended in overtime during the 2023 season.,11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,The player with the highest free throw percentage in the 2020 season (minimum 25 free throws attempted) was Tiffany Mitchell with a free throw percentage of approximately 95.06%.,Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,The Seattle Storm had a win-loss record of 25 wins and 17 losses in the 2022 season.,"25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"The team with the longest winning streak in the 2019 season, inclusive of the postseason, was the Indiana Fever with a winning streak of 1 game.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","Between 2018 and 2024, there were a total of 12 triple-doubles recorded in the WNBA. The players with the most triple-doubles during this period were:\n1. Alyssa Thomas with 12 triple-doubles\n2. Sabrina Ionescu with 4 triple-doubles\n3. Courtney Vandersloot with 2 triple-doubles","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"The team with the best defense in the 2024 season, based on opponent points, was the Connecticut Sun with an average of approximately 71.08 points allowed per game.",Connecticut Sun (opponents held to 71.08 PPG)
8,Identify the game with the highest combined score (both teams) from 2018 to 2024.,The game with the highest combined score (both teams) from 2018 to 2024 was game ID '401558893'. The total combined score of both teams in that game was 540 points.,"07/12/2022, Phoenix Mercury @ Minnesota Lynx (225 points combined)"
9,Which player played the most minutes in the 2021 season?,"Skylar Diggins-Smith played the most minutes in the 2021 season, totaling 1440 minutes.",Skylar Diggins-Smith (1440 minutes)


### Text2SQL (GPT4 Turbo)

In [15]:
gpt4T_text2sql = QAEngine("gpt-4-turbo")
gpt4T_text2sql_result = run_test(gpt4T_text2sql, "text2sql")
gpt4T_text2sql_result

/var/folders/1c/ybx_8cds1ybgdg3z19ptmmk00000gn/T/ipykernel_27648/541717573.py:91: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pyathena.sqlalchemy.rest.AthenaRestDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(connect_str.format(
Failed to execute query.
Traceback (most recent call last):
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/pyathena/common.py", line 575, in _execute
    query_id = retry_api_call(
               ^^^^^^^^^^^^^^^
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/tenacity/__init__.py", line 4

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"The player who scored the most points in a single game between 2018 and 2024 is A'ja Wilson, with a total of 53 points in one game.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"The team with the highest average points per game in the 2021 WNBA season was the Las Vegas Aces, averaging 89.25 points per game.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"In the 2023 WNBA season, there were 11 games that ended in overtime.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"In the 2020 WNBA season, Tiffany Mitchell had the highest free throw percentage among players who attempted at least 25 free throws. She made 77 out of 81 free throws, achieving a free throw percentage of approximately 95.06%.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"In the 2022 season, the Seattle Storm had the following win-loss records:\n\n- In the regular season (STD): 22 wins and 14 losses.\n- In the first round of the playoffs (RD16): 2 wins.\n- In the semifinals (SEMI): 1 win and 3 losses.","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"The team with the longest winning streak in the 2019 WNBA season, including the postseason, was the Washington Mystics, with a streak of 32 consecutive wins.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","Between 2018 and 2024, there were a total of 22 triple-doubles recorded in the WNBA. Alyssa Thomas had the most triple-doubles during this period, recording 12. Other notable players include Sabrina Ionescu with 4 triple-doubles, and Chelsea Gray, Candace Parker, and Courtney Vandersloot each with 2 triple-doubles.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"The team with the best defense in the 2024 WNBA season, measured by the lowest average opponent points per game, was the Sun, with an average of 71.08 points allowed per game.",Connecticut Sun (opponents held to 71.08 PPG)
8,Identify the game with the highest combined score (both teams) from 2018 to 2024.,"The game with the highest combined score between 2018 and 2024 was between Team Wilson and Team Stewart, with a total score of 540 points.","07/12/2022, Phoenix Mercury @ Minnesota Lynx (225 points combined)"
9,Which player played the most minutes in the 2021 season?,"The player who played the most minutes in the 2021 WNBA season was Skylar Diggins-Smith, with a total of 1440 minutes.",Skylar Diggins-Smith (1440 minutes)


### Text2SQL (GPT4o)

In [16]:
gpt4o_text2sql = QAEngine("gpt-4o")
gpt4o_text2sql_result = run_test(gpt4o_text2sql, "text2sql")
gpt4o_text2sql_result

/var/folders/1c/ybx_8cds1ybgdg3z19ptmmk00000gn/T/ipykernel_27648/541717573.py:91: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pyathena.sqlalchemy.rest.AthenaRestDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(connect_str.format(


,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"The player who scored the most points in a single game between 2018 and 2024 is Liz Cambage, who scored 53 points in a game.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"The team with the highest average points per game in the 2021 season was Team WNBA, with an average of 93.0 points per game.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"During the 2023 season, 11 games ended in overtime.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"In the 2020 season, Tiffany Mitchell had the highest free throw percentage with a remarkable 95.06%, having made 77 out of 81 free throw attempts.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,The Seattle Storm had a win-loss record of 25-17 in the 2022 season.,"25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"The team with the longest winning streak in the 2019 season, including the postseason, was the Washington Mystics with a streak of 21 consecutive wins.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","Between 2018 and 2024, there were several players who recorded triple-doubles in the WNBA. The player with the most triple-doubles during this period was Alyssa Thomas, who recorded 12 triple-doubles. Here is a list of players and the number of triple-doubles they recorded:\n\n1. **Alyssa Thomas** - 12 triple-doubles\n2. **Sabrina Ionescu** - 4 triple-doubles\n3. **Courtney Williams** - 2 triple-doubles\n4. **Chelsea Gray** - 2 triple-doubles\n5. **Courtney Vandersloot** - 2 triple-doubles\n6. **Candace Parker** - 2 triple-doubles\n7. **Layshia Clarendon** - 1 triple-double\n8. **Natasha Howard** - 1 triple-double\n9. **Satou Sabally** - 1 triple-double\n10. **Moriah Jefferson** - 1 triple-double\n11. **Sug Sutton** - 1 triple-double\n\nThese players demonstrated exceptional all-around performances to achieve these milestones.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"The team with the best defense in the 2024 season, based on the average opponent points, was the Connecticut Sun, with opponents averaging 71.08 points per game.",Connecticut Sun (opponents held to 71.08 PPG)
8,Identify the game with the highest combined score (both teams) from 2018 to 2024.,"The game with the highest combined score between 2018 and 2024 was between Team Stewart and Team Wilson, with a total score of 270 points.","07/12/2022, Phoenix Mercury @ Minnesota Lynx (225 points combined)"
9,Which player played the most minutes in the 2021 season?,"The player who played the most minutes in the 2021 season was Skylar Diggins-Smith, with a total of 1440 minutes.",Skylar Diggins-Smith (1440 minutes)


### Text2SQL (GPT4o mini)

In [19]:
gpt4omini_text2sql = QAEngine("gpt-4o-mini")
gpt4omini_text2sql_result = run_test(gpt4omini_text2sql, "text2sql")
gpt4omini_text2sql_result

/var/folders/1c/ybx_8cds1ybgdg3z19ptmmk00000gn/T/ipykernel_27648/2419379690.py:91: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pyathena.sqlalchemy.rest.AthenaRestDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(connect_str.format(
Failed to execute query.
Traceback (most recent call last):
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/pyathena/common.py", line 575, in _execute
    query_id = retry_api_call(
               ^^^^^^^^^^^^^^^
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/albertkao/anaconda3/envs/210envtest/lib/python3.11/site-packages/tenacity/__init__.py", line 

,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"The player who scored the most points in a single game between 2018 and 2024 is A'ja Wilson, with a remarkable total of 53 points. This achievement occurred in game ID 401507322.",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"The team with the highest average points per game in the 2021 season was the Las Vegas Aces, with an average of 89.25 points per game.",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"During the 2023 season, there were 11 games that ended in overtime.",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"The player with the highest free throw percentage in the 2020 season, with a minimum of 25 free throws attempted, was Tiffany Mitchell, who achieved a free throw percentage of approximately 95.06%.",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"In the 2022 season, the Seattle Storm had a win-loss record of 25 wins and 17 losses.","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"The team with the longest winning streak in the 2019 season was the Washington Mystics, with a streak of 8 consecutive wins.",Washington Mystics (8 wins)
6,"How many triple-doubles were recorded in the WNBA between 2018 and 2024, and who had the most triple-doubles?","Between 2018 and 2024, there were a total of **29 triple-doubles** recorded in the WNBA. The player with the most triple-doubles during this period was **Alyssa Thomas**, who achieved **12 triple-doubles**.","29 total triple doubles, and Alyssa Thomas had the most (with 12)"
7,Which team had the best defense in the 2024 season (by opponent points)?,"The team with the best defense in the 2024 season, based on opponent points allowed, was the Connecticut Sun, with an average of approximately 71.08 points allowed per game.",Connecticut Sun (opponents held to 71.08 PPG)
8,Identify the game with the highest combined score (both teams) from 2018 to 2024.,"The game with the highest combined score between both teams from 2018 to 2024 was between Team Wilson and Team Stewart, with a total score of 540 points.","07/12/2022, Phoenix Mercury @ Minnesota Lynx (225 points combined)"
9,Which player played the most minutes in the 2021 season?,"The player who played the most minutes in the 2021 season was Skylar Diggins-Smith, with a total of 1,440 minutes played.",Skylar Diggins-Smith (1440 minutes)


### Text2SQL (Claude 3.5)

In [23]:
claude35_text2sql = QAEngine("claude-3-5-sonnet-20240620")
claude35_text2sql_result = run_test(claude35_text2sql, "text2sql")
claude35_text2sql_result

/var/folders/1c/ybx_8cds1ybgdg3z19ptmmk00000gn/T/ipykernel_27648/961702728.py:94: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pyathena.sqlalchemy.rest.AthenaRestDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(connect_str.format(


,question,answer,ground_truths
0,Which player scored the most points in a single game between 2018 and 2024?,"[{'text': ' Based on the query result, I can now answer the question: The player who scored the most points in a single game between 2018 and 2024 was Liz Cambage. She scored an impressive 53 points in a game during the 2018 season. The game ID for this performance is 401018913. This 53-point game by Liz Cambage is actually a WNBA record for the most points scored in a single game. It's a remarkable achievement that showcases her offensive dominance. Cambage set this record on July 17, 2018, while playing for the Dallas Wings against the New York Liberty. Her performance included shooting 17-of-22 from the field and 15-of-16 from the free-throw line, along with 10 rebounds and 5 blocks, making it one of the most dominant individual performances in WNBA history.', 'type': 'text', 'index': 0}]",A’ja Wilson & Liz Cambage (53 points)
1,Which team had the highest average points per game in the 2021 season?,"[{'text': ' Based on the query results, I can now answer the question: The team with the highest average points per game in the 2021 season was the Las Vegas Aces, with an average of 89.25 points per game. This statistic only includes regular season games (as indicated by the 'STD' type in the wnba_schedule table) and represents a strong offensive performance by the Las Vegas Aces throughout the 2021 season.', 'type': 'text', 'index': 0}]",Las Vegas Aces (88.65 PPG)
2,How many games ended in overtime during the 2023 season?,"[{'text': ' Based on the query results, I can now answer the question: During the 2023 WNBA season, there were 11 games that ended in overtime. This means that out of all the games played in the 2023 season, 11 of them went beyond the regular four quarters and required at least one overtime period to determine the winner. Overtime games are often exciting and closely contested matches, showcasing the competitive nature of the WNBA.', 'type': 'text', 'index': 0}]",11 games
3,Which player had the highest free throw percentage in the 2020 season (min. 25 FT’s attempted)?,"[{'text': ' Based on the query results, we can now answer the question: The player with the highest free throw percentage in the 2020 season (with a minimum of 25 free throws attempted) was Tiffany Mitchell. Her statistics are as follows: - Free Throws Made: 77 - Free Throws Attempted: 81 - Free Throw Percentage: 95.06% Tiffany Mitchell demonstrated exceptional accuracy from the free throw line during the 2020 WNBA season, making 77 out of 81 attempts for an impressive 95.06% success rate. This performance placed her at the top of the league for free throw percentage among players who attempted at least 25 free throws that season.', 'type': 'text', 'index': 0}]",Tiffany Mitchell (95.06%)
4,What was the win-loss record of the Seattle Storm in the 2022 season?,"[{'text': ' Based on the query results, I can now answer the question about the Seattle Storm's win-loss record in the 2022 season: The Seattle Storm's win-loss record in the 2022 season was as follows: 1. Regular Season (STD): 22 wins and 14 losses 2. Semi-Finals (SEMI): 1 win and 3 losses 3. First Round (RD16): 2 wins and 0 losses In total, across all game types in the 2022 season, the Seattle Storm had 25 wins and 17 losses. The team performed well in the regular season with a winning record of 22-14. They also swept their first-round playoff series (2-0) but were eliminated in the semi-finals, winning only one game out of four in that round. This record shows that the Seattle Storm had a successful regular season and made it to the playoffs, advancing past the first round before being eliminated in the semi-finals.', 'type': 'text', 'index': 0}]","25 - 17 overall, 22 - 14 regular season"
5,Which team had the longest winning streak in the 2019 season (inclusive of postseason)?,"[{'text': ' Based on the query results, we can answer the question: The team that had the longest win